In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_validData = pd.read_csv("/Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/Data_Sets/Experiment_1_OpenEyes.cvs")

In [3]:

def compute_angvel_summary(df, id_col='SubjectID', val_col='combined_vel'):
    """
    Compute median, IQR, and Bowley coefficient of angular velocity per subject.
    """
    out = (
        df.groupby(id_col)
        .agg(
            Q25=(val_col, lambda x: np.nanquantile(x, 0.25)),
            Median=(val_col, np.nanmedian),
            Q75=(val_col, lambda x: np.nanquantile(x, 0.75))
        )
        .reset_index()
    )
    # Calculate interquartile range and Bowley skew
    out['DistQ1_Q3'] = out['Q75'] - out['Q25']
    out['BowleyCoef'] = (out['Q75'] + out['Q25'] - 2 * out['Median']) / out['DistQ1_Q3']
    
    # Rename columns and drop intermediates
    out = out.rename(columns={
        'Median': 'Combined_AngVel_med',
        'DistQ1_Q3': 'Combined_AngVel_DistQ1_Q3',
        'BowleyCoef': 'Combined_AngVel_bowleyCoef'
    }).drop(columns=['Q25', 'Q75'])

    return out


In [4]:
# 1. All events
df_AngVel_all = compute_angvel_summary(df_validData)

# 2. Saccades only
df_AngVel_saccade = compute_angvel_summary(
    df_validData[df_validData['events'] == -1.0]
)

# 3. Fixations only
df_AngVel_fixation = compute_angvel_summary(
    df_validData[df_validData['events'] == -2.0]
)



/var/folders/wj/11sgsjxd7x75l9tkc6r2l1p80000gn/T/ipykernel_4096/3077874864.py:7: FutureWarning: The provided callable <function nanmedian at 0x1067d1120> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  .agg(
/var/folders/wj/11sgsjxd7x75l9tkc6r2l1p80000gn/T/ipykernel_4096/3077874864.py:7: FutureWarning: The provided callable <function nanmedian at 0x1067d1120> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  .agg(
/var/folders/wj/11sgsjxd7x75l9tkc6r2l1p80000gn/T/ipykernel_4096/3077874864.py:7: FutureWarning: The provided callable <function nanmedian at 0x1067d1120> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" ins

In [5]:
# Save all events
df_AngVel_all.to_csv("/Volumes/TwoTeras/PCA/Data_Sets/Experiment_1/AngularVelocity.csv", index=False)

# Save saccades
df_AngVel_saccade.to_csv("/Volumes/TwoTeras/PCA/Data_Sets/Experiment_1/AngularVelocity_Saccade.csv", index=False)

# Save fixations
df_AngVel_fixation.to_csv("/Volumes/TwoTeras/PCA/Data_Sets/Experiment_1/AngularVelocity_Fixation.csv", index=False)
